# Producción EPIC016 - Isolation Forest

## 1. Importación de librerías

In [11]:
from random import sample

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
import pickle

#Mostrar todas las columnas de un dataframe
pd.set_option('display.max_columns', None)

import warnings

warnings.filterwarnings('ignore')

%matplotlib inline

## 2. Importación data

In [6]:
#Seleccionar ruta de la data
dataframe = pd.read_csv(r'../data/modeling/01_raw.csv', encoding='latin')
dataframe.shape

(52568, 12)

## 3. Feature engineering

In [7]:
#Cambio nombre de columna
dataframe.rename(columns = {'NUMPERIODO':'PERIODO'}, inplace = True)

#Creo variable FLG_ANSLBNP que es la unión de las variables FLG_AN y FLG_LSBNP
#Inicializo la variable en cero
dataframe['FLG_ANLSBNP']=0
#Creo las condicionales donde dependiendo de la  categoría de la variable inical pinto como 1
dataframe.loc[(dataframe.FLG_AN==1) & (dataframe.FLG_LSB_NP==1),'FLG_ANLSBNP' ] = 1
dataframe.loc[(dataframe.FLG_AN==1) & (dataframe.FLG_LSB_NP==0),'FLG_ANLSBNP' ] = 1
dataframe.loc[(dataframe.FLG_AN==0) & (dataframe.FLG_LSB_NP==1),'FLG_ANLSBNP' ] = 1
#Creo variables dummy de los tipos de persona para luego analizarlo en el perfilamiento (No es parte del modelamiento esta variable)
data_dummies=pd.get_dummies(dataframe['TIPPER_AGRUP']).astype(int)
#Agrego las variables dummies creadas al dataframe original
dataframe = pd.concat([dataframe, data_dummies], axis = 1)

dataframe.shape

(52568, 16)

## 3. Universo

In [8]:
#Por ser test, ultimo periodo y monto de ingresos mayor a 1000
dataset=dataframe.loc[(dataframe['PERIODO']==202207) & (dataframe['MTO_INGRESOS']>1000)]
dataset.shape

(5892, 16)

In [9]:
dataset.to_csv(r'../data/modeling/02_test_preprocessed.csv',index=False,sep='|')

In [10]:
#Selecciono solo las variables que necesita el modelo
ds=dataset[['MTO_INGRESOS', 'POR_CASH', 'FLG_PERFIL_3DESVT_TRX', 'FLG_ANLSBNP']]

## 4. Ejecución de modelo

In [14]:
#Cargo modelo:
model_path = r'../src/03_models/IF_MEpic16.model'
loaded_model = pickle.load(open(model_path, "rb"))

In [15]:
#Aplico el modelo cargado sobre la data. Columna Outlier es la que tiene el resultado del modelo
dataset['Outlier']=loaded_model.predict(ds)
dataset.head()

,PERIODO,CODCLAVECIC,TIPPER_AGRUP,MTO_INGRESOS,MTO_EGRESOS,RATIO_ING_TOT,POR_CASH,FLG_PERFIL_3DESVT_TRX,FLG_AN,FLG_LSB_NP,MEDIA_DEP,DESV_DEP,FLG_ANLSBNP,PJ,PN_CN,PN_SN,Outlier
7,202207,654705,PN_SN,195066.42,325352.23,0.37,0.31,0,1,0,483924.06,227490.43,1,0,0,1,1
15,202207,661429,PN_SN,4849.95,18483.54,0.21,0.00,0,0,0,4615.55,6369.96,0,0,0,1,1
37,202207,614713,PN_SN,10021.17,22351.98,0.31,0.00,0,0,0,85813.06,41125.28,0,0,0,1,1
42,202207,622915,PN_CN,66696.89,1576.32,0.98,0.00,0,0,0,53336.22,16484.24,0,0,1,0,1
51,202207,628945,PN_SN,9365.05,1248.76,0.88,0.00,0,0,0,30163.98,23838.18,0,0,0,1,1


In [16]:
#Verifico casos considerados como outlier
dataset.groupby(['Outlier']).size()

Outlier
-1      44
 1    5848
dtype: int64

In [17]:
#Exporto la data con la columna Outlier recién creada
dataset.to_csv(r'../data/modeling/03_test_output.csv',index=False)


#Línea si se necesita separador |
#dataset.to_csv(r'Output_model.csv',index=False,sep='|')

In [19]:
#Registro de versión de python y scikit-learn con la que fue hecho el modelo
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.1.3.
